In [1]:
import json

import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from survhive.cv import regularisation_path, regularisation_path_precond
from survhive.cox import CoxPHElasticNet, CoxPHPrecond
from survhive.utils import transform_survival, transform_preconditioning
import timeit
from survhive.utils import transform_survival, transform_preconditioning
from sklearn.decomposition import PCA
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.util import Surv

In [2]:
with open(f"../config.json") as f:
    config = json.load(f)

In [3]:
elastic_net_timing_one = {cancer: [] for cancer in config["datasets"]}
elastic_net_timing_zero_nine = {cancer: [] for cancer in config["datasets"]}
elastic_net_timing_zero_one = {cancer: [] for cancer in config["datasets"]}

precond_timing_one = {cancer: [] for cancer in config["datasets"]}
precond_timing_zero_five = {cancer: [] for cancer in config["datasets"]}
precond_timing_zero = {cancer: [] for cancer in config["datasets"]}

pipe_efron = make_pipeline(
    StandardScaler(),
    PCA(n_components=16),
    CoxPHSurvivalAnalysis(ties="efron")
)

In [4]:
cancer = "BLCA"
data = pd.read_csv(f"../data/processed/TCGA/{cancer}_data_preprocessed.csv").iloc[:, 1:]
X = StandardScaler().fit_transform(data.iloc[:, 3:])
y = data.iloc[:, :2]
pipe_efron.fit(data.iloc[:, 3:], Surv().from_arrays(time=y["OS_days"].values, event=y["OS"].values))
eta_hat = pipe_efron.predict(data.iloc[:, 3:])

In [6]:
regularisation_path(
    X=pd.DataFrame(X)
    .iloc[
        np.argsort(y["OS_days"], kind="stable"),
    ]
    .to_numpy(),
    y=transform_survival(
        time=y["OS_days"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
        event=y["OS"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
    ),
    X_test=pd.DataFrame(X)
    .iloc[
        np.argsort(y["OS_days"], kind="stable"),
    ]
    .to_numpy(),
    model=CoxPHElasticNet(
       tie_correction="efron", 
        alpha=0.0,
        l1_ratio=1.0,

        warm_start=True,
        n_irls_iter=5,
        tol=0.0001,
        verbose=0,
        inner_solver_max_iter = 100,
        inner_solver_max_epochs = 50000,
        inner_solver_p0 = 10,
        inner_solver_prune = True,
        check_global_kkt=True,
    ),
            l1_ratio=1.0,
            eps=0.05,
            n_alphas=100,
            alphas=None,
            max_first=True
)


Alphas: 1/100
Alphas: 2/100
Alphas: 3/100
Alphas: 4/100
Alphas: 5/100
Alphas: 6/100
Alphas: 7/100
Alphas: 8/100
Alphas: 9/100
Alphas: 10/100
Alphas: 11/100
Alphas: 12/100
Alphas: 13/100
Alphas: 14/100
Alphas: 15/100
Alphas: 16/100
Alphas: 17/100
Alphas: 18/100
Alphas: 19/100
Alphas: 20/100
Alphas: 21/100
Alphas: 22/100
Alphas: 23/100
Alphas: 24/100
Alphas: 25/100
Alphas: 26/100
Alphas: 27/100
Alphas: 28/100
Alphas: 29/100
Alphas: 30/100
Alphas: 31/100
Alphas: 32/100
Alphas: 33/100
Alphas: 34/100
Alphas: 35/100
Alphas: 36/100
Alphas: 37/100
Alphas: 38/100
Alphas: 39/100
Alphas: 40/100
Alphas: 41/100
Alphas: 42/100
Alphas: 43/100
Alphas: 44/100
Alphas: 45/100
Alphas: 46/100
Alphas: 47/100
Alphas: 48/100
Alphas: 49/100
Alphas: 50/100
Alphas: 51/100
Alphas: 52/100
Alphas: 53/100
Alphas: 54/100
Alphas: 55/100
Alphas: 56/100
Alphas: 57/100
Alphas: 58/100
Alphas: 59/100
Alphas: 60/100
Alphas: 61/100
Alphas: 62/100
Alphas: 63/100
Alphas: 64/100
Alphas: 65/100
Alphas: 66/100
Alphas: 67/100
Alph

In [ ]:
regularisation_path_precond(
    X=pd.DataFrame(X)
    .iloc[
        np.argsort(y["OS_days"], kind="stable"),
    ]
    .to_numpy(),
    y=transform_preconditioning(
        time=y["OS_days"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
        event=y["OS"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
        y_teacher=eta_hat
    ),
    X_test=pd.DataFrame(X)
    .iloc[
        np.argsort(y["OS_days"], kind="stable"),
    ]
    .to_numpy(),
    model=CoxPHPrecond(
       tie_correction="efron", 
        alpha=0.0, 
        tau=0.0, 
        maxiter=1000,
        rtol=1e-6,
        verbose=0,
        default_step_size=1.0,
        check_global_kkt=True
    ),
    tau=1.0,
    eps=0.05,
    n_alphas=100,
    max_first=True,
)


In [ ]:
for cancer in config["datasets"]:
    data = pd.read_csv(f"../data/processed/TCGA/{cancer}_data_preprocessed.csv").iloc[:, 1:]
    X = StandardScaler().fit_transform(data.iloc[:, 3:])
    y = data.iloc[:, :2]
    pipe_efron.fit(data.iloc[:, 3:], Surv().from_arrays(time=y["OS_days"].values, event=y["OS"].values))
    eta_hat = pipe_efron.predict(data.iloc[:, 3:])
    
    # Elastic net timings
    for i in range(10):
        try:
            start = timeit.default_timer()

            regularisation_path(
                X=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                y=transform_survival(
                    time=y["OS_days"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                    event=y["OS"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                ),
                X_test=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                model=CoxPHElasticNet(
                    alpha=0.0,
                    l1_ratio=1.0,

                    warm_start=True,
                    n_irls_iter=5,
                    tol=0.0001,
                    verbose=0,
                    tie_correction="efron",
                    inner_solver_max_iter = 100,
                    inner_solver_max_epochs = 50000,
                    inner_solver_p0 = 10,
                    inner_solver_prune = True,
                    check_global_kkt=True,
                ),
                l1_ratio=1.0,
                eps=0.05,
                n_alphas=100,
                alphas=None,
                max_first=True
            )
            end = timeit.default_timer()
            elastic_net_timing_one[cancer].append(end - start)
        except:
            elastic_net_timing_one[cancer].append(np.nan)
    for i in range(10):
        try:
            start = timeit.default_timer()

            regularisation_path(
                X=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                y=transform_survival(
                    time=y["OS_days"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                    event=y["OS"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                ),
                X_test=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                model=CoxPHElasticNet(
                    alpha=0.0,
                    l1_ratio=0.9,

                    warm_start=True,
                    n_irls_iter=5,
                    tol=0.0001,
                    verbose=0,
                    tie_correction="efron",
                    inner_solver_max_iter = 100,
                    inner_solver_max_epochs = 50000,
                    inner_solver_p0 = 10,
                    inner_solver_prune = True,
                    check_global_kkt=True,
                ),
                l1_ratio=0.9,
                eps=0.05,
                n_alphas=100,
                alphas=None,
                max_first=True
            )
            end = timeit.default_timer()
            elastic_net_timing_zero_nine[cancer].append(end - start)
        except:
            elastic_net_timing_zero_nine[cancer].append(np.nan)
    for i in range(10):
        try:
            start = timeit.default_timer()

            regularisation_path(
                X=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                y=transform_survival(
                    time=y["OS_days"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                    event=y["OS"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                ),
                X_test=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                model=CoxPHElasticNet(
                    alpha=0.0,
                    l1_ratio=0.1,

                    warm_start=True,
                    n_irls_iter=5,
                    tol=0.0001,
                    verbose=0,
                    tie_correction="efron",
                    inner_solver_max_iter = 100,
                    inner_solver_max_epochs = 50000,
                    inner_solver_p0 = 10,
                    inner_solver_prune = True,
                    check_global_kkt=True,
                ),
                l1_ratio=0.1,
                eps=0.05,
                n_alphas=100,
                alphas=None,
                max_first=True
            )
            end = timeit.default_timer()
            elastic_net_timing_zero_one[cancer].append(end - start)
        except:
            elastic_net_timing_zero_one[cancer].append(np.nan)
    
    # Preconditioning timings
    for i in range(10):
        try:
            start = timeit.default_timer()

            regularisation_path_precond(
                X=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                y=transform_preconditioning(
                    time=y["OS_days"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                    event=y["OS"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                    y_teacher=eta_hat
                ),
                X_test=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                model=CoxPHPrecond(
                   tie_correction="efron", 
                    alpha=0.0, 
                    tau=1.0, 
                    maxiter=1000,
                    rtol=1e-6,
                    verbose=0,
                    default_step_size=1.0,
                    check_global_kkt=True
                ),
                tau=1.0,
                eps=0.05,
                n_alphas=100,
                max_first=True,
            )

            end = timeit.default_timer()
            precond_timing_one[cancer].append(end - start)
        except:
            precond_timing_one[cancer].append(np.nan)
    for i in range(10):
        try:
            start = timeit.default_timer()

            regularisation_path_precond(
                X=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                y=transform_preconditioning(
                    time=y["OS_days"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                    event=y["OS"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                    y_teacher=eta_hat
                ),
                X_test=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                model=CoxPHPrecond(
                   tie_correction="efron", 
                    alpha=0.0, 
                    tau=0.5, 
                    maxiter=1000,
                    rtol=1e-6,
                    verbose=0,
                    default_step_size=1.0,
                    check_global_kkt=True
                ),
                tau=0.5,
                eps=0.05,
                n_alphas=100,
                max_first=True,
            )

            end = timeit.default_timer()
            precond_timing_zero_five[cancer].append(end - start)
        except:
            precond_timing_zero_five[cancer].append(np.nan)
    for i in range(10):
        try:
            start = timeit.default_timer()

            regularisation_path_precond(
                X=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                y=transform_preconditioning(
                    time=y["OS_days"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                    event=y["OS"].to_numpy()[np.argsort(y["OS_days"], kind="stable")],
                    y_teacher=eta_hat
                ),
                X_test=pd.DataFrame(X)
                .iloc[
                    np.argsort(y["OS_days"], kind="stable"),
                ]
                .to_numpy(),
                model=CoxPHPrecond(
                   tie_correction="efron", 
                    alpha=0.0, 
                    tau=0.0, 
                    maxiter=1000,
                    rtol=1e-6,
                    verbose=0,
                    default_step_size=1.0,
                    check_global_kkt=True
                ),
                tau=0.0,
                eps=0.05,
                n_alphas=100,
                max_first=True,
            )

            end = timeit.default_timer()
            precond_timing_zero[cancer].append(end - start)
        except:
            precond_timing_zero[cancer].append(np.nan)
    

pd.DataFrame(elastic_net_timing_one).to_csv(
    "../results/elastic_net_timing_one.csv", index=False
)
pd.DataFrame(elastic_net_timing_zero_nine).to_csv(
    "../results/elastic_net_timing_zero_nine.csv", index=False
)
pd.DataFrame(elastic_net_timing_zero_one).to_csv(
    "../results/elastic_net_timing_zero_one.csv", index=False
)

pd.DataFrame(precond_timing_zero).to_csv(
    "../results/precond_timing_zero.csv", index=False
)
pd.DataFrame(precond_timing_zero_five).to_csv(
    "../results/precond_timing_zero_five.csv", index=False
)
pd.DataFrame(precond_timing_one).to_csv(
    "../results/precond_timing_one", index=False
)